In [1]:
import pandas as pd

In [2]:
pwd

'/root'

In [1]:
# plan and preprocesstion
# d = 220
# fpe = DatasetfingerPrintExtractor
# npfp = num of preprocess used for fingerprint extraction
# use kits23 dataset to train
from batchgenerators.utilities.file_and_folder_operations import *
import shutil
dataset_id = 220
num_process = 8
verbose = False
# def fingerprint_extractor_class(dataset_id, num_process, verbose):
# trans data to imageTr, labelsTr 
input_folder = 'autodl-tmp/kits19-cnn-master/kits23-main/dataset'
out_base = 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw'
imagesTr = join(out_base, "imagesTr")
labelsTr = join(out_base, "labelsTr")

# make dir
maybe_mkdir_p(imagesTr)
maybe_mkdir_p(labelsTr)
# print(imagesTr)

# require case head files
# copy cases[0,8]
cases = subdirs(input_folder, prefix='case_', join=False)

cases_to_process = cases[:9]
for tr in cases_to_process:
    shutil.copy(join(input_folder, tr, 'imaging.nii.gz'), join(imagesTr, f'{tr}_0000.nii.gz'))
    shutil.copy(join(input_folder, tr, 'segmentation.nii.gz'), join(labelsTr, f'{tr}.nii.gz'))
# num_training_cases=len(cases)

In [3]:
# generate data json
from typing import Tuple, Union, List


# 对labels中的字典类型进行判断
labels = {
    "background":0,
    "kidney":(1,2,3),
    "masses":(2,3),
    "tumor": 2
}
# print(labels.values()) #dict_values([0, (1, 2, 3), (2, 3), 2])
has_regions: bool = any([isinstance(i, (tuple,list)) and len(i) > 1 for i in labels.values()])
# print(has_regions) #Ture

# 标签的排列顺序
region_class_order=(1,3,2)

# 数据集的模态信息转换成channel_name = {['0':'CT'}
channels_name={0:"CT"}
keys = list(channels_name.keys())
# print(keys)#[0]
for k in keys:
#     print(k)
    if not isinstance(k, str):
#         print("error")
        channels_name[str(k)] = channels_name[k]
        del channels_name[k]
# print(channels_name)#{'0': 'CT'}

# 对labels中的字典类型进行判断,并转换成整数。这个要求字典类型为整数
for l in labels.keys():
#     print(l)
    value = labels[l]
#   print(values)
    if isinstance(value, (tuple, list)):
        value = tuple([int(i) for i in value])
        labels[l] = value
        print(value)
    else:
        labels[l] = int(labels[l])
        print(labels[l])
# print(labels)
file_ending='.nii.gz'
license: str = 'although the person is lazy, but i think the code is already very good!'
converted_by: str= 'i dont know who write this code'
dataset_name = "KITS2023"
reference = 'none'
release = '0.1.3'
description = "KITS2023"
overwrite_image_reader_writer = 'NibabelIOWithReoient'
input_folder = 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw'
cases = subdirs(input_folder, prefix='case_', join=False)
num_training_cases=len(cases)
dataset_json = {
    'channel_names':channels_name,
    'labels':labels,
    'numTraining':num_training_cases,
    'file_ending':file_ending,
    'licence':license,
    'converted_by':converted_by
}
if dataset_name is not None:
    dataset_json['name'] = dataset_name
if reference is not None:
    dataset_json['reference']=reference
if release is not None:
    dataset_json['release']=release
if description is not None:
    dataset_json['description']=description
if overwrite_image_reader_writer is not None:
    dataset_json['overwrite_image_reader_writer']=overwrite_image_reader_writer
if region_class_order is not None:
    dataset_json['region_class_order']=region_class_order

# dataset_json.update(kwargs)
# print(dataset_json)
out_base = 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw'
save_json(dataset_json, join(out_base, 'dataset_json'), sort_keys=False)


0
(1, 2, 3)
(2, 3)
2


In [4]:
# dataset finger print
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
from multiprocessing import Pool
import re
num_process = 8
out_base = "autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw"
dataset_json = load_json("autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/dataset_json")
# dataset = 
folder = join(out_base, "imagesTr")
files = subfiles(folder, suffix=dataset_json["file_ending"], join=False, sort=True)
# print(files)
# ['case_00000_0000.nii.gz', 'case_00001_0000.nii.gz', 'case_00002_0000.nii.gz', 'case_00003_0000.nii.gz', 'case_00004_0000.nii.gz', 'case_00005_0000.nii.gz', 'case_00006_0000.nii.gz'
crop = len(file_ending) + 5
files1 = [i[:-crop] for i in files]
# ['case_00000', 'case_00001', 'case_00002', 'case_00003', 'case_00004', 'case_00005', 'case_00006', 'case_00007', 'case_00008']
# print(files)

# identifier
files2 = np.unique(files1)
# print(files)

param_list = [(folder, files, file_ending, f) for f in files2]
# print(param_list)
list_of_lists = []

# 匹配正则表达式筛选文件
def create_paths_fn(folder, files, file_ending, f):
    p = re.compile(re.escape(f) + r"_\d\d\d\d" + re.escape(file_ending))
    return [join(folder, i) for i in files if p.fullmatch(i)]

# images
with Pool(processes=num_process) as pool:
    list_of_lists = pool.starmap(create_paths_fn, param_list)
#     print(list_of_lists)

segs = [join(out_base, 'labelsTr', i + dataset_json['file_ending']) for i in files2]
# print(segs)

dataset = {i: {'images': im, 'label': se} for i, im, se in zip(files2, list_of_lists, segs)}
print(dataset)

{'case_00000': {'images': ['autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/imagesTr/case_00000_0000.nii.gz'], 'label': 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/labelsTr/case_00000.nii.gz'}, 'case_00001': {'images': ['autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/imagesTr/case_00001_0000.nii.gz'], 'label': 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/labelsTr/case_00001.nii.gz'}, 'case_00002': {'images': ['autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/imagesTr/case_00002_0000.nii.gz'], 'label': 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/labelsTr/case_00002.nii.gz'}, 'case_00003': {'images': ['autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/imagesTr/case_00003_0000.nii.gz'], 'label': 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/labelsTr/case_00003.nii.gz'}, 'case_00004': {'images': ['autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/imagesTr/case_00004_0000.nii.gz'], 'label': 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw

In [5]:
pip install acvl-utils==0.2

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 4.3 MB 5.4 MB/s eta 0:00:01
  Created wheel for acvl-utils: filename=acvl_utils-0.2-py3-none-any.whl size=22468 sha256=2530c14cef72f46d1f45bb941f0a05b4544ae31230e608a26aa534b5ce1eb31a
  Stored in directory: /root/.cache/pip/wheels/6a/60/75/83067c05dd96f4860a844060174dcb7880f53ebfdfd559061d
Successfully built acvl-utils
Note: you may need to restart the kernel to use updated packages.


In [9]:
# 前景体素数量
from batchgenerators.utilities.file_and_folder_operations import *
from typing import Tuple, Union, List
from typing import Type
from nibabel.orientations import io_orientation, axcodes2ornt, ornt_transform
import multiprocessing
import nibabel
from concurrent.futures import ProcessPoolExecutor
from typing import List
from scipy.ndimage import binary_fill_holes
from acvl_utils.cropping_and_padding.bounding_boxes import get_bbox_from_mask, bounding_box_to_slice
import numpy as np
num_process = 8
preprocessed_output_folder = 'autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_preprocessed'
num_foreground_voxels_for_intensitystats = 10e7
properties_file = join(preprocessed_output_folder, 'dataset_fingerprint.json')

overwrite_existing = False

class NibabelIOWithReoient():
    def read_images(self, image_fnames: Union[List[str], Tuple[str, ...]]) -> Tuple[np.ndarray,dict]:
        images = []
        original_affines = []
        reoriented_affines = []
        spacings_for_nnunet = []
        for f in image_fnames:
#           加载医学图像
            nib_image = nibabel.load(f)
            assert nib_image.ndim == 3, 'only 3d images'
#           记录原始的仿射矩阵，为了之后数据处理记录原始的内容，包含体素在三维空间的坐标，方向，间距（每个体素的大小），偏移量
            original_affine = nib_image.affine
#           重定向，从原始仿射变换的方向转换成标准方向。
            reoriented_image = nib_image.as_reoriented(io_orientation(original_affine))
            reoriented_affine = reoriented_image.affine
            
            original_affines.append(original_affine)
            reoriented_affines.append(reoriented_affine)
            
#           获取体素间距。header包含图像元数据，get_zooms()获得每个轴的体素间距，[::-1]将nibabel的（x,y,z）改为simpleITK的轴顺序（z,y,x）  
            spacings_for_nnunet.append(
                [float(i) for i in reoriented_image.header.get_zooms()[::-1]]
            )
#           调整图像数据的轴顺序get_fdata表示获取图像的numpy格式的数据，transpose是将轴的顺序转换成（z,y,x），[None]表示在开始添加一个新的维度[1,z,y,x]
            images.append(reoriented_image.get_fdata().transpose((2,1,0))[None])

            dict = {
                'nibabel_stuff':{
                    'original_affine':original_affines[0],
                    'reoriented_affine':reoriented_affines[0],
                },
                'spacing':spacings_for_nnunet[0]
            }
        return np.vstack(images, dtype=np.float32, casting='unsafe'), dict
    
#     return a array
    def read_seg(self, seg_fname: str) -> Tuple[np.ndarray, dict]:
        return self.read_images((seg_fname,))
    
#     将分割结果保存为一个nifTI文件
    def wirte_seg(self, seg: np.ndarray, output_fname: str, properties: dict) -> None:
#       一个numpy数组
        seg = seg.transpose((2,1,0)).astype(np.uint8 if np.max(seg) else np.uint16, copy=False)
#       创建NIFTI图像对象，seg调整轴顺序的分割结果，affine是使用图像重新定向后的仿射矩阵。变成一样的，seg和img是一样的仿射变换
        seg_nib = nibabel.NiftilImage(seg, affine=properties['nibabel_stuff']['reoriented_affine'])
        
#       计算原始图像的仿射矩阵
        img_ornt = io_orientation(properties['nibabel_stuff']['original_affine'])
    
#         将方向代码（RAS）转换为方向矩阵
        ras_ornt = axcodes2ornt("RAS")
# 计算从标准方向（RAS）到原始方向的转换矩阵。ras_ornt是标准方向矩阵，img_ornt是原始方向矩阵
        from_canonical = ornt_transform(ras_ornt, img_ornt)
#        将分割结果重新定向到原始方向
        seg_nib_reoriented = seg_nib.as_reoriented(form_canonical)
    
#       验证恢复后的方向是否正确
        if not np.allclose(properties['nibabel_stuff']['original_affine'],seg_nib_reoriented.affine):
            print(f'warning: restored affine does not match original affine. File:{output_fname}')
            print(f'original affine\n', properyies['nibabel_stuff']['original_affine'])
            print(f'restored affine\n', seg_nib_reoriented.affine)
#       将重新定向的分割结果保存为NIFTI文件，文件路径为output_fname
        nibabel.save(seg_nib_reoriented, out_fname)

    

    
def determine_reader_writer_from_dataset_json(dataset_json_content: dict, example_file: str=None,
                                              allow_normatching_filename: bool=False, verbose:bool=True):
#     获取overwrite_image_reader_writer的键值
    if 'overwrite_image_reader_writer' in dataset_json_content.keys() and \
            dataset_json_content['overwrite_image_reader_writer'] != 'None':
        ioclass_name = dataset_json_content['overwrite_image_reader_writer']
        print(ioclass_name)
#     return ioclass_name
#         try:
#             ret 是用来确定含有NibabelIOWithReoient这个函数的

#       这里返回一个读写器类，如果rw中的read_images成功执行，就会返回这个rw类别
    if ioclass_name == 'NibabelIOWithReoient':
#         Ni = NibabelIOWithReoient()
#         no gpu is not work
#         a = Ni.read_images((example_file,))
#         print(a)
        return NibabelIOWithReoient #返回类本身，而不是类的实例    
    
def analyze_case(image_files: List[str], segmentation_file: str, reader_writer_class: Type,
                num_samples: int = 10000):
    rw = reader_writer_class()
    images, properties_images = rw.read_images(image_files)
    print(images.shape)
    print("hello")
    segmentation, properties_seg = rw.read_seg(segmentation_file)

#裁剪图像和分割标签，创建一个非零掩码
#总结：下面这段代码是使用非零掩码，去裁剪图像和标签，裁剪的图像和标签大小是一致的，相当于去除掉冗余部分，只保留组织
    assert images.ndim in (3, 4), "data must have shape (C,X,Y,Z) or shape (C,X,Y)"
    #data=(C,X,Y,Z), data[0]=(X,Y,Z),判断所有数据都不为0，结果是一个布尔数组，形状与data[0]一样,不为0的为True，为0的为False
    non_zero_mask = images[0] != 0
    print(non_zero_mask)
    #data.shape[0] is C,这里就是将每个通道中的True都放在一个矩阵里面，只要有true，这个位置就是true
    for c in range(1, images.shape[0]):
        #将通道c的非0区域合并到nonzero_mask中
        non_zero_mask |= images[c] !=0
        
    #填充二值图像中的孔洞，孔洞☞True里面的False区域，True包围的里面有false就会填充为true
    non_zero_mask = binary_fill_holes(non_zero_mask)
    
    #从非零掩码中获取包含所有非零区域的的最小边界框 ，结果是bbox = (1,4,1,4,1,4)x轴从1到4，y轴从1到4,z轴从1到4
    bbox = get_bbox_from_mask(non_zero_mask)
    
    #将边界框转换为切片器，用于裁剪数据，结果是（slice(1,4), slice(1,4)）
    slicer = bounding_box_to_slice(bbox)
    #使用切片器裁剪非零掩码，并添加一个额外的维度，以匹配通道维度,切片后的形状为(1,3,3),这里面还是掩码，就是True或false
    nonzero_mask = nonzero_mask[slicer][None]
    
    #sclice data 使用切片数据裁剪输入数据data，slice(None)用于保留所有通道，裁剪后的data的形状为（c，3，3）
    slicer = (slice(None), ) + slicer
    images = images[slicer]
    
    #使用切片数据裁剪分割数据，将分割标签中为0 且不在非零掩码内的区域设置为nonzero_label
    if segmentation is not None:
        segmentation = segmentation[slicer]
        #裁剪分割的=0的部分并且掩码部分为false，就会打上nonzero_label标签
        segmentation[(segmentation == 0) & (~nonzero_mask)] = nonzero_label
    else:
        segmentation = np.where(nonzero_mask, np.int8(0), np.int8(nonzero_label))
        
#从图像中收集前景区域的强度值，并计算每个通道的强度统计信息
    
       
# print(dataset[dataset.keys().__iter__().__next__()]['images'][0])
# autodl-tmp/kits19-cnn-master/kits23-main/nnUNet_raw/imagesTr/case_00000_0000.nii.gz
if not isfile(properties_file) or overwrite_existing:
    reader_writer_class = determine_reader_writer_from_dataset_json(dataset_json,dataset[dataset.keys().__iter__().__next__()]['images'][0])
#     print(reader_writer_class[0])
# 用于强度统计，也就是判断每个案例的像素数量，强度统计是计算像素的像素值，这里是数量，不是像素值
    num_forground_samples_per_case = int(num_foreground_voxels_for_intensitystats // len(dataset))
    print("num_foreground:",num_forground_samples_per_case)
    print('len dataset:',len(dataset))
    print('len num_foreground_voxels:',int(num_foreground_voxels_for_intensitystats))
#     print(dataset[dataset.keys().__iter__().__next__()]['images'][0])
    futures = []
    with ProcessPoolExecutor(max_workers=2) as executor:
        for k in dataset.keys():
#             print(k)
            futures.append(executor.submit(analyze_case, dataset[k]['images'],
                                    dataset[k]['label'], reader_writer_class, num_forground_samples_per_case))
#             print(r)
# 获取每个任务的结果
        for future in futures:
            try:
                result = future.result()  # 等待任务完成并获取结果
                print(result)  # 打印结果
            except Exception as e:
                print(f"Task failed with error: {e}")
#     with multiprocessing.get_context("spawn").Pool(8) as p:
#         for k in dataset.keys():
# #             print(k)#case_00000,case_00001
#             print(dataset[k]['images'])
# #     异步任务提交到进程池（Pool），用于异步执行DatasetFingerPrintExtractor.analyze_case
# # p用于管理进程池，starmap_async异步执行函数，返回一个asyncresult对象，表示异步任务的结果。
#             r.append(p.starmap_async(analyze_case,((dataset[k]['images'],dataset[k]['label'],reader_writer_class,
#                                                    num_forground_samples_per_case),)))
            
   

NibabelIOWithReoient
num_foreground: 11111111
len dataset: 9
len num_foreground_voxels: 100000000
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Task failed with error: A process in the process pool was terminated 